# 处理数据，生成一个大的npy文件
导入os、numpy、PIL库，并确保tqdm库可用。

In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [8]:
# 文件夹路径
image_dir = 'data/Img'
mask_dir = 'data/Lab'

# 获取文件列表
images = sorted(os.listdir(image_dir))
masks = sorted(os.listdir(mask_dir))

# 确保文件对应
assert len(images) == len(masks), "图片和掩码数量不一致"

# 完整路径
image_paths = [os.path.join(image_dir, img) for img in images]
mask_paths = [os.path.join(mask_dir, msk) for msk in masks]

# 分割数据集
train_imgs, test_imgs, train_masks, test_masks = train_test_split(
    image_paths, mask_paths, test_size=0.2, random_state=42
)

def load_images(paths):
    data = []
    for path in paths:
        img = Image.open(path).convert('RGB')
        img_array = np.array(img)
        img_array = np.transpose(img_array, (2, 0, 1))  # 将通道轴移到最前面
        data.append(img_array)
    return np.array(data)

def load_masks(paths):
    data = []
    for path in paths:
        img = Image.open(path).convert('L')
        data.append(np.array(img))
    return np.array(data)

# 加载并保存训练集
train_images = load_images(train_imgs)
train_masks = load_masks(train_masks)
np.save('data/train_images.npy', train_images)
np.save('data/train_masks.npy', train_masks)

# 加载并保存测试集
test_images = load_images(test_imgs)
test_masks = load_masks(test_masks)
np.save('data/test_images.npy', test_images)
np.save('data/test_masks.npy', test_masks)

# 运行

In [9]:
!python main.py

Parameters:
{'work_dir': './work_dir/', 'config': './config/train.yaml', 'phase': 'train', 'save_score': False, 'seed': 1, 'feeder': 'feeder.Feeder', 'num_worker': 8, 'train_feeder_args': {'mode': 'train', 'data_path': './data/train_images.npy', 'label_path': './data/train_masks.npy'}, 'test_feeder_args': {'mode': 'test', 'data_path': './data/test_images.npy', 'label_path': './data/test_masks.npy'}, 'model': 'net.self_net', 'model_args': {'num_class': 4, 'channel': 3}, 'weights': None, 'ignore_weights': [], 'base_lr': 0.01, 'step': [20, 40, 60], 'device': 0, 'optimizer': 'Adam', 'nesterov': True, 'batch_size': 16, 'test_batch_size': 16, 'start_epoch': 37, 'num_epoch': 150, 'weight_decay': 1e-05}

模型参数量: 0.10 M
Epoch:[38/150]
100%|███████████████████████████████████████████| 40/40 [00:05<00:00,  7.77it/s]
Train iou: [0.30507429 0.66557104 0.54106291] Train miou: 0.5039027478218588 Mean loss: 0.25275725275278094 LR: [0.0014239574134637463]
[[4.3192846e+07 4.9595600e+05 9.5033800e+05 2.65

清理log

In [7]:
!rm -rf work_dir/*